In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import sys
from sklearn.model_selection import train_test_split

In [2]:
# Read the CSV file using pandas
data_frame = pd.read_csv('./data/restaurant-menus.csv')

# Convert the data frame to a NumPy array
numpy_array = data_frame.to_numpy()

# Print the NumPy array
print(numpy_array)

[[1 'Extra Large Pizza' 'Extra Large Meat Lovers' 'Whole pie.'
  '15.99 USD']
 [1 'Extra Large Pizza' 'Extra Large Supreme' 'Whole pie.' '15.99 USD']
 [1 'Extra Large Pizza' 'Extra Large Pepperoni' 'Whole pie.' '14.99 USD']
 ...
 [40227 'Beverages' 'Iced Tea' nan '1.95 USD']
 [40227 'Desserts' 'Sesame Balls (4 pcs)' nan '3.95 USD']
 [40227 'Desserts' 'Fried Biscuits (4 pcs)' nan '3.95 USD']]


In [3]:
original_tags = np.unique(numpy_array[:, 1])
count_tags = len(original_tags)
indexes_to_remove = []
for i in range(count_tags):
    if original_tags[i][0].upper() not in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
        indexes_to_remove.append(i)
tags = np.delete(original_tags, indexes_to_remove)
for tag in original_tags:
    print(tag)

    APPETIZER 
   A la Carte
  Appetizer Soup &amp; Salads
  Appetizers
  CALZONES  
  Chicken Salads
  Chicken Sammies
  Chinese &amp; Japanese Combo (Buffet dine in open now!!!)
  Classics Sammies
  Fried Rice &amp; Lo Mein Noodles
  Fruit Tea 鮮果茶
  Milk Tea 奶茶
  Nashville Hot Chicken Sliders &amp; Tenders
  Premium Sammies
  Steak Salads
  Steak Sammies
  🌄 DESAYUNOS(BREAKFAST)DESDE LAS 7:00AM-12:00pm
 "RA"MEN 
 (Preparados) 
 1/3lb Thickburgers
 2 LITER SODA
 A La Carte (Or make your own combination for $10.50) (4 Different Items)
 Acompañamientos | Sides
 Add-on
 Appetizers
 Appetizers (Aperitivos)
 Artisan Ice Cream 
 Asian Style Dessert
 Assorted Donuts
 Assorted Fancy Rolls.
 BEVERAGES
 BREAKFAST
 BREAKFAST &amp; LUNCH TO GO ITEMS 
 BUILD YOUR OWN BURGER
 BURGER / SANDWICHES
 BURGERS
 BURRITO
 Bakery 
 Banh Mi
 Beans/legumes/ Lentils 
 Beef Burgers
 Beverage
 Beverages
 Beverages 
 Beverages (Bebida)
 Beverages (饮料)
 Biryanis
 Biscuit Meals - All Day Breakfast
 Boneless W/ Frie

Soup / Noodle /Fried Rice
Soup / Salad
Soup / Salad 
Soup / Salads
Soup / Salads / Snacks
Soup / Sopas
Soup / Stew
Soup / Udon
Soup And Salad
Soup And Salad 
Soup And Salads
Soup Anytime
Soup Appetizer
Soup Appetizers
Soup Bar
Soup Base 汤底
Soup Based Entrees
Soup Based Noodles and Rice Noodles 汤面及米粉
Soup Bases
Soup Bowl
Soup Bowls
Soup By The Cup
Soup Choice
Soup Combos
Soup D
Soup Dishes
Soup Dishes (Mi &amp; Hu Tieu)
Soup Dishes (Side of Fried rice IS NOT available)
Soup Du Jour
Soup Dumplings
Soup Dumplings 小笼包
Soup Entrees
Soup In Bowl
Soup Items
Soup Kettle and Combos
Soup Lover
Soup Market Soups and Fresh Salads
Soup N Chili
Soup N' Salads
Soup Noodle
Soup Noodle 汤面
Soup Noodles
Soup Noodles 汤拉面
Soup Of The Day
Soup Online Menu
Soup Salad Side
Soup Served
Soup Specials
Soup Specials - Sup Khac
Soup Specialties
Soup Stew
Soup Udon
Soup Udon Noodles
Soup Vermicelli and Noodle 汤粉面类
Soup Yourself
Soup and / or Salad
Soup and Appetizer
Soup and Appetizers
Soup and Caldo
Soup and Chili

In [4]:
#We're going to generate a transcation data that has rating and the tag
#For every 10% data, user will also buy the same menu from that subset.
#Also, user will also able to buy random from outside the data 
#[id_user, tag, rating]
c_data = 250
c_transaction = int(5e5)
c_user = int(1e4)
print(f"Expected Sparse : {c_transaction / (c_user * c_data) * 100} %")

Expected Sparse : 20.0 %


In [5]:
used_tags = np.sort(np.random.choice(tags, size=c_data, replace=False))
for tag in used_tags:
    if tag[-1] == "\n":
        tag = tag[:-1]
    print(tag)

A La Carte - Chicken
ADDITIONAL SIDE DISHES
Add-on Items
Additional Vegetarian Entrees
Al La Cluck Desserts
Angus Burgers (1/2 lb)
Anytime Small Bite
Appetizer Combinations
Appetizers, Soups, and Salads
Appetizers,Small Bites
Appetizers-Khai Vi
Asian Small Plates
Aunt Ollie’s Sides
Aussie Special Pies
Authentic Chinese Food 正宗中餐
BBQ of the Border
BEER &amp; WINE
BOGO
BREAKFAST FOR BEGINNERS
BURRITOS &amp; CHICKENS
Barbecue and Grill/  All order come with naan, chickpeas and salad 
Baskets 
Beef Entrees (Bò)
Beverage (lt)
Beverages (Nuoc Giat Khat)
Beverages PM
Bhatura and Poori
Bibimbaps
Big Bowl
Bitcoin Pizza - 14"
Bloody Mary &amp; Mimosa Kits
Blue Shell Mussels (Prince Edward Island)
Bocaditos
Boiled
Bougey's Brunch Menu
Breakfast Classics
Breakfast Croissants or Muffins
Buns
Carne
Carne y pollo
Carvel Ice Cream &amp; Treats
Chaat, Pav, Cholle
Chef'S Special
Chef's Selections
Chicken Temptations
Chicken Wings, Pizza Rolls, Bread Sticks 
Chocolate Toffee
Choose Your Meat
Classic Favo

In [31]:
i = 0
ten_percent = c_data // 10
transaction = np.empty((c_transaction, 3), dtype=int)
while i < c_transaction:
    user = random.randint(1, c_user)
    food = random.randint(user//ten_percent, user//ten_percent + ten_percent)
    if random.random() <= 0.2:
        food = random.randint(1, c_data) #Change food
        rating = random.randint(4,5)
    else:
        rating = random.randint(1,5-c_user//(c_user // 3))
    transaction[i][0] = user
    transaction[i][1] = food
    transaction[i][2] = rating
    i += 1
transaction

array([[3897,  179,    2],
       [8664,  368,    1],
       [9670,  406,    2],
       ...,
       [5598,  195,    4],
       [8742,  374,    1],
       [   2,   10,    1]])

In [32]:
#Randomly generated transaction
df = pd.DataFrame(transaction)
df.columns = ["User_id", "Food_id", "Rating"]
df.head()

,User_id,Food_id,Rating
0,3897,179,2
1,8664,368,1
2,9670,406,2
3,5415,224,2
4,245,14,1


In [8]:
#highest average rated food
agg_ratings = df.groupby('Food_id').agg(mean_rating = ('Rating', 'mean'),
                                                number_of_ratings = ('Rating', 'count')).reset_index()
agg_ratings.sort_values(by="mean_rating", ascending=False)

,Food_id,mean_rating,number_of_ratings
1,1,3.196581,468
2,2,3.074951,507
3,3,2.892193,538
4,4,2.864912,570
6,6,2.745370,648
...,...,...,...
414,414,1.464674,368
424,424,1.461538,39
261,261,1.458669,992
423,423,1.367647,68


In [9]:
#Most popular food
agg_ratings.sort_values(by="number_of_ratings", ascending=False)

,Food_id,mean_rating,number_of_ratings
143,143,2.077023,1532
139,139,2.123399,1483
138,138,2.058824,1479
76,76,2.130494,1479
234,234,2.068428,1476
...,...,...,...
422,422,1.619469,113
423,423,1.367647,68
0,0,1.341463,41
424,424,1.461538,39


In [10]:
matrix = df.pivot_table(index='User_id', columns='Food_id', values='Rating')
print(sys.getsizeof(matrix) / 1024 / 1024)
matrix.head()

32.577537536621094


Food_id,0,1,2,3,4,5,6,7,8,9,...,416,417,418,419,420,421,422,423,424,425
User_id,,,,,,,,,,,,,,,,,,,,,
1,2.0,1.666667,NaN,2.0,2.0,NaN,2.0,NaN,1.666667,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.5,1.500000,NaN,NaN,NaN,1.666667,1.8,1.666667,2.000000,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,2.000000,1.0,NaN,2.0,1.333333,2.0,1.500000,1.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,NaN,2.0,2.0,2.0,NaN,NaN,2.000000,1.000000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,1.000000,1.0,NaN,1.5,1.500000,NaN,1.000000,2.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# if a restaurant has no transaction, it will not appear below
all_col = list(matrix.columns)
print(all_col)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [12]:
#it will become the input shape
print(matrix.shape)

(10000, 426)


In [13]:
train_val, test = train_test_split(matrix, test_size = 0.1, random_state = 69)
train, val = train_test_split(matrix, test_size = 0.1, random_state = 69)


In [14]:
df

,User_id,Food_id,Rating
0,7008,295,2
1,3494,154,1
2,3628,158,2
3,667,99,5
4,6114,247,1
...,...,...,...
499995,2660,108,1
499996,2278,93,2
499997,4492,196,1
499998,5610,239,1


In [36]:
# Data Preprocessing


n_users = df['User_id'].nunique() + 1
n_items = df['Food_id'].nunique() + 1

user_ids = df['User_id'].values
item_ids = df['Food_id'].values
ratings = df['Rating'].values

# Splitting into Training and Test sets
train_user_ids, test_user_ids, train_item_ids, test_item_ids, train_ratings, test_ratings = train_test_split(
    user_ids, item_ids, ratings, test_size=0.2, random_state=42
)

# Define the model architecture
latent_dim = 5
regularization = 0.01

user_input = tf.keras.layers.Input(shape=(1,))
item_input = tf.keras.layers.Input(shape=(1,))

user_embedding = tf.keras.layers.Embedding(n_users, latent_dim, input_length=1,
                                           embeddings_regularizer=tf.keras.regularizers.l2(regularization))(user_input)
item_embedding = tf.keras.layers.Embedding(n_items, latent_dim, input_length=1,
                                           embeddings_regularizer=tf.keras.regularizers.l2(regularization))(item_input)

user_vec = tf.keras.layers.Flatten()(user_embedding)
item_vec = tf.keras.layers.Flatten()(item_embedding)

concatenated_vec = tf.keras.layers.Concatenate()([user_vec, item_vec])
output = tf.keras.layers.Dense(1)(concatenated_vec)

model = tf.keras.Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')

# Model Training
model.fit([train_user_ids, train_item_ids], train_ratings, epochs=10, batch_size=100)

# Model Evaluation
test_loss = model.evaluate([test_user_ids, test_item_ids], test_ratings)
print("Test Loss:", test_loss)

Epoch 1/10
4000/4000 [==============================] - 5s 1ms/step - loss: 2.2011
Epoch 2/10
4000/4000 [==============================] - 5s 1ms/step - loss: 1.6103
Epoch 3/10
4000/4000 [==============================] - 5s 1ms/step - loss: 1.5872
Epoch 4/10
4000/4000 [==============================] - 5s 1ms/step - loss: 1.5765
Epoch 5/10
4000/4000 [==============================] - 5s 1ms/step - loss: 1.5602
Epoch 6/10
4000/4000 [==============================] - 5s 1ms/step - loss: 1.5490
Epoch 7/10
4000/4000 [==============================] - 4s 1ms/step - loss: 1.5366
Epoch 8/10
4000/4000 [==============================] - 5s 1ms/step - loss: 1.5200
Epoch 9/10
4000/4000 [==============================] - 4s 1ms/step - loss: 1.4969
Epoch 10/10
3125/3125 [==============================] - 3s 961us/step - loss: 1.4786
Test Loss: 1.4786287546157837


In [39]:
# Making Recommendations
user_ids_unique = np.unique(test_user_ids)
# user_ids_unique = [6969]

recommendations = []

for user_id in user_ids_unique:
    user_ratings = df[(df['User_id'] == user_id)]['Rating']
    sorted_items = np.argsort(user_ratings)[::-1]  # Sort items based on ratings in descending order
    top_items = item_ids_unique[sorted_items][:3]  # Top 3 recommendations
    recommendations.append((user_id, top_items))

print("Recommendations:")
for user_id, items in recommendations:
    print("User", user_id, ":", items)

Recommendations:
User 1 : [201  49  70]
User 2 : [101 320 163]
User 3 : [401 344 406]
User 4 : [115 147 158]
User 5 : [115 224 373]
User 6 : [ 44 163 117]
User 7 : [ 39 368 314]
User 8 : [ 65 163  40]
User 9 : [179 101  96]
User 10 : [352 158 112]
User 11 : [125 234  86]
User 12 : [158 406 262]
User 13 : [115  79  18]
User 14 : [373 406 368]
User 15 : [210  39 158]
User 16 : [210 106  86]
User 17 : [137  37 344]
User 18 : [179  74 189]
User 19 : [ 53 211  81]
User 20 : [179  14  65]
User 21 : [221 320  70]
User 22 : [261 117 406]
User 23 : [352 398  96]
User 24 : [401 163  86]
User 25 : [179 211 234]
User 26 : [373 114 115]
User 27 : [ 63 137 147]
User 28 : [158 401 221]
User 29 : [146  79 407]
User 30 : [137  40 320]
User 31 : [368 406 224]
User 32 : [ 44 407  14]
User 33 : [171  81 114]
User 34 : [ 40  39 189]
User 35 : [398  49 344]
User 36 : [345 117 368]
User 37 : [ 87 262 139]
User 38 : [210 320  65]
User 39 : [ 87 224  49]
User 40 : [336  65  81]
User 41 : [ 18 171  63]
User 42 

User 5060 : [373  79  37]
User 5061 : [139  70 179]
User 5062 : [ 96 234 211]
User 5063 : [ 87 171 101]
User 5064 : [234  44  14]
User 5065 : [ 87 352  18]
User 5066 : [406 201  70]
User 5067 : [ 81 243 171]
User 5068 : [407 171  37]
User 5069 : [295 163  49]
User 5070 : [368 147 171]
User 5071 : [210 243  16]
User 5072 : [ 96 179  70]
User 5073 : [179  94  14]
User 5074 : [101 201 115]
User 5075 : [ 53 171 115]
User 5076 : [189 210  94]
User 5077 : [79 39 87]
User 5078 : [ 44 171  70]
User 5079 : [112 201  63]
User 5080 : [183  57 406]
User 5081 : [ 87  63 158]
User 5082 : [ 16 163 101]
User 5083 : [243  79 210]
User 5084 : [137  81  14]
User 5085 : [49 44 86]
User 5086 : [ 53 114 221]
User 5087 : [368 171 158]
User 5088 : [243 401  94]
User 5089 : [320  44 137]
User 5090 : [44 14 87]
User 5091 : [ 65  53 163]
User 5092 : [ 16  14 314]
User 5093 : [234 171  65]
User 5094 : [ 40  14 379]
User 5095 : [ 39 406 163]
User 5096 : [179 115  86]
User 5097 : [ 14 201 314]
User 5098 : [146 224 